<a href="https://colab.research.google.com/github/anishdulal/Naive-Bayes-Sentiment-Analysis/blob/master/SentimentAnalysisNaiveBayes18thMarch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Naive Bayes

In [0]:
#load necessary libraries
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#Read Dataset
#df=pd.read_csv("sentiment.csv")
#df.head()
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
#open('/content/gdrive/My Drive/sentiment.csv')
df = pd.read_csv(open('/content/gdrive/My Drive/sentiment.csv'))
#df = pd.read_csv('sentiment.csv')
df.head()

,Phrase,label
0,Spider-man is better than any summer blockbust...,Positive
1,of good sense,Positive
2,", then knock yourself out and enjoy the big sc...",Negative
3,So could young romantics out on a date .,Positive
4,humour,Positive


In [4]:
len(df[df['label']=='Positive']) #total number of positive class

1800

In [5]:
len(df[df['label']=='Negative']) #total number of negative class

1000

## Data Cleaning 

In [0]:
def encode_labels(dataset):
    if dataset=="Positive":
        dataset=1
    elif dataset=="Negative":
        dataset=0
    return dataset 

df['label']=df['label'].apply(encode_labels)

In [7]:
df.head()

,Phrase,label
0,Spider-man is better than any summer blockbust...,1
1,of good sense,1
2,", then knock yourself out and enjoy the big sc...",0
3,So could young romantics out on a date .,1
4,humour,1


## Splitting dataset

In [8]:
total_len=len(df)
train_len=int(0.6*total_len)
test_len=int(0.4*total_len)
print(total_len,train_len,test_len)

2800 1680 1120
ERROR! Session/line number was not unique in database. History logging moved to new session 59


In [0]:
train_df=df[0:train_len]
test_df=df[train_len:]

In [0]:
vocab=list()
for i in train_df.Phrase.tolist():
    vocab.extend(i.split(" "))      
unique_vocab=list(set(vocab))

In [11]:
print(len(vocab),len(unique_vocab))

13855 1844


## Calculating Priors


In [12]:
class_priors=train_df['label'].value_counts(normalize=True)
class_priors

1    0.646429
0    0.353571
Name: label, dtype: float64

## Getting Negative class words

In [0]:
class_0_words=[]
for sentence in train_df[train_df['label']==0]['Phrase']:
    word_list=sentence.lower().split(" ")
    new_word_list=[]
    for w in word_list:
        if w!='':
            new_word_list.append(w)   
    class_0_words.extend(new_word_list)

## Getting Positive class words

In [0]:
class_1_words=[]
for sentence in train_df[train_df['label']==1]['Phrase']:
    word_list=sentence.lower().split(" ")
    new_word_list=[]
    for w in word_list:
        if w!='':
            new_word_list.append(w)   
    class_1_words.extend(new_word_list)

In [0]:
class_words={
    0:class_0_words,
    1:class_1_words
}

## Finding word probabilities 
Alpha is the smoothing factor. It is addition of small number in order to compensate the effect of words which are not in the vocabulary.           

In [0]:
def count_occurence(word,word_list):
    count=0
    for w in word_list:
        if word==w:
            count+=1
    return count   

def get_word_probs_with_laplacian_smoothing(class_0_words,class_1_words,alpha,unique_vocab):
    word_probs={
        0:{},
        1:{}
    }

    for word in unique_vocab:
        word_probs[0][word]=((count_occurence(word,class_0_words) + alpha)/(len(class_0_words)+alpha*len(unique_vocab)))
        word_probs[1][word]=((count_occurence(word,class_1_words) + alpha)/(len(class_1_words)+alpha*len(unique_vocab)))
    
    return word_probs


## PREDICTION

In [0]:
def predict(alpha,dataset): 
    word_probs=get_word_probs_with_laplacian_smoothing(class_0_words,class_1_words,alpha,unique_vocab)
    predicted_class=[]
    for phrase in dataset:
        phraseArr=phrase.split(" ")
        prob_0_given_string=1
        prob_1_given_string=1
        for w in phraseArr:
            try:
                prob_0_given_string=prob_0_given_string*word_probs[0][w]
                prob_1_given_string=prob_1_given_string*word_probs[1][w]
            except Exception:
                w_prob_class_0=((alpha/len(class_0_words)+alpha*len(unique_vocab)))
                prob_0_given_string=prob_0_given_string*w_prob_class_0
                w_prob_class_1=((alpha/len(class_1_words)+alpha*len(unique_vocab)))
                prob_1_given_string=prob_1_given_string*w_prob_class_1
            prob_0_given_string*=class_priors[0]
            prob_1_given_string*=class_priors[1]
           
        if prob_1_given_string>prob_0_given_string:
            predicted_class.append(1)
        else:
            predicted_class.append(0)    
    
    return predicted_class

In [0]:
predictions=predict(1,test_df["Phrase"])

In [0]:

def get_confusion_matrix(actual,predicted):
    confusion_matrix=np.zeros((2,2))
    for x,y in zip(actual,predicted):
        confusion_matrix[x,y] +=1
    return confusion_matrix

In [20]:
confusion_matrix=get_confusion_matrix(test_df['label'],predictions)
confusion_matrix

array([[230., 176.],
       [ 28., 686.]])

In [21]:
#accuracy
accuracy=(confusion_matrix[0][0]+confusion_matrix[1][1])/np.sum(confusion_matrix)
accuracy

0.8178571428571428

In [22]:
#Recall
recall=(confusion_matrix[0][0])/(confusion_matrix[0][0]+confusion_matrix[0][1])
recall

0.5665024630541872

In [23]:
#Precision
precision=(confusion_matrix[0][0])/(confusion_matrix[0][0]+confusion_matrix[1][0])
precision

0.8914728682170543

In [24]:
#F1-Score
f1_score=(2*recall*precision)/(recall+precision)
f1_score

0.6927710843373495